# osmnx package

In [ ]:
## explore the OSMnx package

# pip install contextily

import osmnx as ox
import folium
import contextily as cx
import matplotlib.pyplot as plt

In [ ]:
# This is using the Nominatim API to find the place by name and geocode it.
# Nominatim is the search engine that powers the search bar on openstreetmap.org.

PLACE_NAME = 'Charlottenburg, Charlottenburg-Wilmersdorf, Berlin, Germany'

In [ ]:
## Show Charlottenburg road network

G = ox.graph_from_place(PLACE_NAME, network_type='drive')
ox.plot_graph(G) 

In [ ]:
# calculate the area of charlottenburg
charlottenburg = ox.gdf_from_place(PLACE_NAME)
charlottenburg_proj = ox.project_gdf(charlottenburg)
area = charlottenburg_proj['geometry'].area[0]  # in sq meters
print(area)

In [ ]:
## plot the roads on a basemap
ox.plot_graph_folium(G)

In [ ]:
ox.graph_to_gdfs(G, nodes=False).explore()

In [ ]:
## plot the bike network on a basemap

# G = ox.graph_from_place(PLACE_NAME, network_type='bike')
# type(G)
# ox.plot_graph_folium(G)

In [ ]:
## create edge and node dataframes from road graph object
area,edges = ox.graph_to_gdfs(G)
area.head(2)

In [ ]:
edges.head(2)

In [ ]:
area.explore()

In [ ]:
## extract benches from the area

benches = ox.geometries_from_place(PLACE_NAME, tags={'amenity':'bench'})
benches.head(2)

In [ ]:
type(benches)

In [ ]:
## extract parks from the area

parks = ox.geometries_from_place(PLACE_NAME, tags={'leisure':'park'})
parks.head();
# parks

In [ ]:
## extract water fountains from the area

water_fountains = ox.geometries_from_place(PLACE_NAME, tags={'amenity':'drinking_water'})
water_fountains.head()


In [ ]:
import geopandas as gpd
type(water_fountains)

In [ ]:
water_fountains.geometry[0]

In [ ]:
# display the water fountains on a map
m = folium.Map(location=[52.516667, 13.4], zoom_start=13)
folium.GeoJson(water_fountains).add_to(m)
m

In [ ]:
# calculate the distance between water_fountains, osmid 449162399 and 449162395
water_fountains.geometry[0].distance(water_fountains.geometry[1])

In [ ]:
# calculate distance between water_fountains[0] and benches[0]
water_fountains.geometry[0].distance(benches.geometry[0])

In [ ]:
# calculate the area of parks[0]
parks.geometry[0].area

In [ ]:
## extract fuel stations from the area

fuel_stations = ox.geometries_from_place(PLACE_NAME, {"amenity": "fuel"})
fuel_stations.head();

In [ ]:
## show all the fuel stations on the map

area_crs = area.to_crs('3857')
edges_crs = edges.to_crs('3857')
fuel_stations_crs = fuel_stations.to_crs('3857')

fig, ax = plt.subplots(figsize=(10, 14))
area_crs.plot(ax=ax, facecolor='white')
edges_crs.plot(ax=ax, linewidth=1, edgecolor='blue', alpha=0.5)
fuel_stations_crs.plot(ax=ax, color='yellow', alpha=0.9, markersize=22)
plt.tight_layout()
cx.add_basemap(ax,crs=area_crs.crs.to_string())

In [ ]:
## show all the parks on the map

area_crs = area.to_crs('3857')
edges_crs = edges.to_crs('3857')
parks_crs = parks.to_crs('3857')

fig, ax = plt.subplots(figsize=(10, 14))
area_crs.plot(ax=ax, facecolor='white')
edges_crs.plot(ax=ax, linewidth=1, edgecolor='blue', alpha=0.5)
parks_crs.plot(ax=ax, color='yellow', alpha=0.9, markersize=22)
plt.tight_layout()
cx.add_basemap(ax,crs=area_crs.crs.to_string())

In [ ]:
## show the benches on the map

area_crs = area.to_crs('3857')
edges_crs = edges.to_crs('3857')
benches_crs = benches.to_crs('3857')

fig, ax = plt.subplots(figsize=(10, 14))
area_crs.plot(ax=ax, facecolor='white')
edges_crs.plot(ax=ax, linewidth=1, edgecolor='blue', alpha=0.5)
benches_crs.plot(ax=ax, color='yellow', alpha=0.9, markersize=22)
plt.tight_layout()
cx.add_basemap(ax,crs=area_crs.crs.to_string())



In [ ]:
## geocode_to_gdf allows you to use nominatim api to geocode a place name.
## https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.geocoder.geocode_to_gdf

origin = (
    ox.geocode_to_gdf("zoologischer garten")
    .to_crs(edges.crs)  
    .at[0, "geometry"]  
    .centroid  
)

destination = (
    ox.geocode_to_gdf("Charlottenburg, Berlin, Germany")
    .to_crs(edges.crs)
    .at[0, "geometry"]
    .centroid 
)

origin_node_id = ox.nearest_nodes(G, origin.x, origin.y)
destination_node_id = ox.nearest_nodes(G, destination.x, destination.y)

In [ ]:
route = ox.shortest_path(G, origin_node_id, destination_node_id)
route;

In [ ]:
## create a route passes all the water fountains
route = ox.shortest_path(G, origin_node_id, destination_node_id, weight="length")
route;

In [ ]:
ox.plot_route_folium(G, route, route_linewidth=6, node_size=0)
# ox.plot_route_folium(G, extended_route, route_linewidth=6, node_size=0)

# overpy package

In [ ]:
# pip install overpy

## running a single query without osmnx
## use overpy to extract all the benches in a grid located around DSR

import overpy
api = overpy.Overpass()
result = api.query("""
            [out:json];
                node["amenity"="bench"](52.495,13.32,52.51,13.35);
            out body;
            >;
            out skel qt;
            """)

len(result.nodes)
## 222 benches in this area

# nominatim API

In [ ]:
## use nominatim through osmnx

# Geocode the location
location = ox.geocode('Charlottenburg, Berlin, Germany')

print(location)
# (52.515747, 13.3096834)